<a href="https://colab.research.google.com/github/golammostafa13/hate_speech_detection/blob/main/k_fold_proposed_model_Bangla.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv1D
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
import numpy as np

2022-04-03 05:11:38.331199: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-03 05:11:38.331276: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import one_hot,Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense , Flatten ,Embedding,Input,LSTM, Bidirectional, Dense, Dropout,GRU,SpatialDropout1D, Conv1D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from tensorflow.keras.initializers import Constant
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [ ]:
file = 'content/cleaned_data.csv'
data = pd.read_csv(file, encoding='utf-8')

In [ ]:
data.head()

,Unnamed: 0,sentence,hate,category,cleaned_text,sentenceLength,ReviewChars
0,0,যত্তসব পাপন শালার ফাজলামী!!!!!,1,sports,যত্তসব পাপন শালার ফাজলামী,4,30
1,1,পাপন শালা রে রিমান্ডে নেওয়া দরকার,1,sports,পাপন শালা রে রিমান্ডে নেওয়া দরকার,6,33
2,2,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,1,sports,জিল্লুর রহমান স্যারের ছেলে এতো বড় জারজ হবে এটা...,20,107
3,3,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,1,sports,শালা লুচ্চা দেখতে পাঠার মত দেখা যায়,7,35
4,4,তুই তো শালা গাজা খাইছচ।তুর মার হেডায় খেলবে সাকিব,1,sports,তুই তো শালা গাজা খাইছচ তুর মার হেডায় খেলবে সাকিব,10,48


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_text'], data['hate'],random_state = 0,test_size = 0.20)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((24000,), (6000,), (24000,), (6000,))

In [ ]:
corpus=[]
words = 0;
j = 0
for i in data['cleaned_text'].values:
    corpus.append(str(i).split(" "))
    words += len(corpus[j])
    j += 1
print("Total words = ", words)
print("Total Sentances = ", len(corpus))

Total words =  509593
Total Sentances =  30000


In [ ]:
import gensim
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
import os

# tokenize every words so that evey words maps to numaric value
tok = Tokenizer()
tok.fit_on_texts(data['cleaned_text'].values.astype('U'))
encd_rev = tok.texts_to_sequences(data['cleaned_text'].values.astype('U'))
# tok.word_index

# make all the input sentance same length with add padding
max_rev_len = 60 # max lenght of a sentance
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim = 100 # embedding dimension 
pad_rev= pad_sequences(encd_rev, maxlen=60, padding='post')

print("Size of vocabulary : ", vocab_size)
print("All sentances with same length ", pad_rev.shape)

# lebel encode the output label to categorical
Y = data['hate']
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# # convert integers to dummy variables (i.e. one hot encoded)
# dummy_y = np_utils.to_categorical(encoded_Y)

# print(encoder.classes_)
# print(dummy_y)

# test train split
X_train,X_test,y_train,y_test=train_test_split(pad_rev,encoded_Y,test_size=0.2, random_state = 0)


# make a dictionary. word as key and feature vector as value
embedding_index={}
f = open('content/ban_hate_w2v.txt',encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:])
    embedding_index[word]=coefs
f.close()

# create a embeddings matrix with 200 dimenstion
EMBEDDING_DIM=100
embedding_matrix=np.zeros((vocab_size,EMBEDDING_DIM))
for word, i in tok.word_index.items():
    if i>vocab_size:
        continue
    embedding_vector=embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

/home/mostafa/env/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Size of vocabulary :  47752
All sentances with same length  (30000, 60)


In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((24000, 60), (6000, 60), (24000,), (6000,))

In [ ]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
print(y_train.shape, y_test.shape)

(24000, 1) (6000, 1)


In [ ]:
# Model configuration
batch_size = 64
no_classes = 100
no_epochs = 5
optimizer = Adam()
verbosity = 1
num_folds = 10

In [ ]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

In [ ]:
inputs.shape, targets.shape

((30000, 60), (30000, 1))

In [ ]:
from keras.layers import Layer
from keras import backend as K
class attention(Layer):
    def __init__(self,**kwargs):
        super(attention,self).__init__(**kwargs)

    def build(self,input_shape):
        self.W=self.add_weight(name="att_weight",shape=(input_shape[-1],1),initializer="normal")
        self.b=self.add_weight(name="att_bias",shape=(input_shape[1],1),initializer="zeros")        
        super(attention, self).build(input_shape)

    def call(self,x):
        et=K.squeeze(K.tanh(K.dot(x,self.W)+self.b),axis=-1)
        at=K.softmax(et)
        at=K.expand_dims(at,axis=-1)
        output=x*at
        return K.sum(output,axis=1)

    def compute_output_shape(self,input_shape):
        return (input_shape[0],input_shape[-1])

    def get_config(self):
        return super(attention,self).get_config()

# proposedModel(word2vec)

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
  model=Sequential()
  embedding_layer=Embedding(vocab_size,
                        EMBEDDING_DIM,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=60,
                        trainable=False)  
  model.add(embedding_layer)
  model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
  model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
  model.add(attention())
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.25))
  model.add(Dense(256, activation='relu'))
  model.add(Dense(1,activation='sigmoid'))
  model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  # model.summary()

  # Generate a print
  print('------------------------------------------------------------------------')
  print(f'Training for fold {fold_no} ...')

  # Fit data to model
  history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)

  # Generate generalization metrics
  scores = model.evaluate(inputs[test], targets[test], verbose=0)
  print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
  acc_per_fold.append(scores[1] * 100)
  loss_per_fold.append(scores[0])
  # Increase fold number
  fold_no = fold_no + 1

  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
  print('------------------------------------------------------------------------')
  print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

2022-03-26 14:52:02.740884: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-26 14:52:02.740996: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-26 14:52:02.741095: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mostafa): /proc/driver/nvidia/version does not exist
2022-03-26 14:52:02.851077: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
422/422 [==============================] - 498s 1s/step - loss: 0.4040 - accuracy: 0.8219
Epoch 2/5
422/422 [==============================] - 485s 1s/step - loss: 0.3512 - accuracy: 0.8483
Epoch 3/5
422/422 [==============================] - 534s 1s/step - loss: 0.3289 - accuracy: 0.8616
Epoch 4/5
422/422 [==============================] - 539s 1s/step - loss: 0.3145 - accuracy: 0.8684
Epoch 5/5
422/422 [==============================] - 498s 1s/step - loss: 0.3016 - accuracy: 0.8713
Score for fold 1: loss of 0.32966700196266174; accuracy of 86.43333315849304%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
422/422 [==============================] - 577s 1s/step - loss: 0.4083 - accuracy: 0.8167
Epoch 2/5
422/422 [==============================] - 626s 1s/step - loss: 0.3552 - accuracy: 0.8457
Epoch 3/5
422/422 [=========

In [ ]:
import gensim
from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split
import os

# tokenize every words so that evey words maps to numaric value
tok = Tokenizer()
tok.fit_on_texts(data['cleaned_text'].values.astype('U'))
encd_rev = tok.texts_to_sequences(data['cleaned_text'].values.astype('U'))
# tok.word_index

# make all the input sentance same length with add padding
max_rev_len = 60 # max lenght of a sentance
vocab_size = len(tok.word_index) + 1  # total no of words
embed_dim = 100 # embedding dimension 
pad_rev= pad_sequences(encd_rev, maxlen=60, padding='post')

print("Size of vocabulary : ", vocab_size)
print("All sentances with same length ", pad_rev.shape)

# lebel encode the output label to categorical
Y = data['hate']
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# # convert integers to dummy variables (i.e. one hot encoded)
# dummy_y = np_utils.to_categorical(encoded_Y)

# print(encoder.classes_)
# print(dummy_y)

# test train split
X_train,X_test,y_train,y_test=train_test_split(pad_rev,encoded_Y,test_size=0.2, random_state = 0)


# make a dictionary. word as key and feature vector as value
embedding_index={}
f = open('content/bn_glove.39M.100d.txt',encoding='utf-8')
for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:])
    embedding_index[word]=coefs
f.close()

# create a embeddings matrix with 200 dimenstion
EMBEDDING_DIM=100
embedding_matrix=np.zeros((vocab_size,EMBEDDING_DIM))
for word, i in tok.word_index.items():
    if i>vocab_size:
        continue
    embedding_vector=embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i]=embedding_vector

/home/mostafa/env/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


Size of vocabulary :  47752
All sentances with same length  (30000, 60)


In [ ]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)
print(y_train.shape, y_test.shape)

(24000, 1) (6000, 1)


In [ ]:
# Model configuration
batch_size = 64
no_classes = 100
no_epochs = 5
optimizer = Adam()
verbosity = 1
num_folds = 10

In [ ]:
# Define per-fold score containers
acc_per_fold = []
loss_per_fold = []

# Merge inputs and targets
inputs = np.concatenate((X_train, X_test), axis=0)
targets = np.concatenate((y_train, y_test), axis=0)

# proposedModel(glove)

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
    model=Sequential()
    embedding_layer=Embedding(vocab_size,
                        EMBEDDING_DIM,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=60,
                        trainable=False)  
    model.add(embedding_layer)
    model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
    model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
    model.add(attention())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  # model.summary()

  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)
    model_path = "content/models_word2vec/" + str(fold_no) + "model"
    
    # convert the history.history dict to a pandas DataFrame:     
    hist_df = pd.DataFrame(history.history) 

    # save to json:  
    hist_json_file = "content/History_word2vec/" + str(fold_no) + 'history.json' 
    with open(hist_json_file, mode='w') as f:
        hist_df.to_json(f)
            
    #save model
    model.save(model_path)
    
  # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
  # Increase fold number
    fold_no = fold_no + 1

  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
422/422 [==============================] - 407s 924ms/step - loss: 0.4060 - accuracy: 0.8199
Epoch 2/5
422/422 [==============================] - 382s 906ms/step - loss: 0.3538 - accuracy: 0.8487
Epoch 3/5
422/422 [==============================] - 389s 920ms/step - loss: 0.3314 - accuracy: 0.8594
Epoch 4/5
422/422 [==============================] - 383s 909ms/step - loss: 0.3146 - accuracy: 0.8672
Epoch 5/5
422/422 [==============================] - 384s 911ms/step - loss: 0.3030 - accuracy: 0.8718
INFO:tensorflow:Assets written to: content/models_word2vec/1model/assets


INFO:tensorflow:Assets written to: content/models_word2vec/1model/assets


Score for fold 1: loss of 0.32331371307373047; accuracy of 86.16666793823242%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
422/422 [==============================] - 405s 920ms/step - loss: 0.4096 - accuracy: 0.8148
Epoch 2/5
422/422 [==============================] - 388s 920ms/step - loss: 0.3561 - accuracy: 0.8473
Epoch 3/5
422/422 [==============================] - 386s 915ms/step - loss: 0.3321 - accuracy: 0.8583
Epoch 4/5
422/422 [==============================] - 386s 915ms/step - loss: 0.3158 - accuracy: 0.8679
Epoch 5/5
422/422 [==============================] - 387s 917ms/step - loss: 0.3048 - accuracy: 0.8686
INFO:tensorflow:Assets written to: content/models_word2vec/2model/assets


INFO:tensorflow:Assets written to: content/models_word2vec/2model/assets


Score for fold 2: loss of 0.30470460653305054; accuracy of 87.8666639328003%
------------------------------------------------------------------------
Training for fold 3 ...
Epoch 1/5
422/422 [==============================] - 401s 917ms/step - loss: 0.4045 - accuracy: 0.8175
Epoch 2/5
422/422 [==============================] - 386s 914ms/step - loss: 0.3561 - accuracy: 0.8483
Epoch 3/5
422/422 [==============================] - 388s 920ms/step - loss: 0.3305 - accuracy: 0.8583
Epoch 4/5
422/422 [==============================] - 386s 914ms/step - loss: 0.3157 - accuracy: 0.8673
Epoch 5/5
422/422 [==============================] - 388s 921ms/step - loss: 0.3051 - accuracy: 0.8694
INFO:tensorflow:Assets written to: content/models_word2vec/3model/assets


...

Score for fold 8: loss of 0.3152604103088379; accuracy of 87.1999979019165%
------------------------------------------------------------------------
Training for fold 9 ...
Epoch 1/5
422/422 [==============================] - 400s 909ms/step - loss: 0.4076 - accuracy: 0.8180
Epoch 2/5
422/422 [==============================] - 386s 915ms/step - loss: 0.3547 - accuracy: 0.8484
Epoch 3/5
422/422 [==============================] - 385s 913ms/step - loss: 0.3347 - accuracy: 0.8566
Epoch 4/5
422/422 [==============================] - 383s 908ms/step - loss: 0.3165 - accuracy: 0.8672
Epoch 5/5
422/422 [==============================] - 375s 889ms/step - loss: 0.3037 - accuracy: 0.8700
INFO:tensorflow:Assets written to: content/models_word2vec/9model/assets


INFO:tensorflow:Assets written to: content/models_word2vec/9model/assets


Score for fold 9: loss of 0.3276544511318207; accuracy of 86.46666407585144%
------------------------------------------------------------------------
Training for fold 10 ...
Epoch 1/5
422/422 [==============================] - 442s 1s/step - loss: 0.4073 - accuracy: 0.8199
Epoch 2/5
422/422 [==============================] - 418s 991ms/step - loss: 0.3531 - accuracy: 0.8515
Epoch 3/5
422/422 [==============================] - 475s 1s/step - loss: 0.3291 - accuracy: 0.8605
Epoch 4/5
422/422 [==============================] - 421s 997ms/step - loss: 0.3157 - accuracy: 0.8669
Epoch 5/5
422/422 [==============================] - 372s 883ms/step - loss: 0.3018 - accuracy: 0.8733
INFO:tensorflow:Assets written to: content/models_word2vec/10model/assets


INFO:tensorflow:Assets written to: content/models_word2vec/10model/assets


Score for fold 10: loss of 0.3311815857887268; accuracy of 86.19999885559082%
------------------------------------------------------------------------
Score per fold
------------------------------------------------------------------------
> Fold 1 - Loss: 0.3173454701900482 - Accuracy: 86.56666874885559%
------------------------------------------------------------------------
> Fold 2 - Loss: 0.32625967264175415 - Accuracy: 86.56666874885559%
------------------------------------------------------------------------
> Fold 3 - Loss: 0.32167190313339233 - Accuracy: 86.56666874885559%
------------------------------------------------------------------------
> Fold 4 - Loss: 0.32052621245384216 - Accuracy: 86.83333396911621%
------------------------------------------------------------------------
> Fold 5 - Loss: 0.3238447308540344 - Accuracy: 86.76666617393494%
------------------------------------------------------------------------
> Fold 6 - Loss: 0.319640576839447 - Accuracy: 86.79999709

# Bi-GRU(word2vec)

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
    model=Sequential()
    embedding_layer=Embedding(vocab_size,
                        EMBEDDING_DIM,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=60,
                        trainable=False)  
    model.add(embedding_layer)
    model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
#     model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
#     model.add(attention())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  # model.summary()

  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)
#     model_path = "content/models_word2vec/" + str(fold_no) + "model"
    
#     # convert the history.history dict to a pandas DataFrame:     
#     hist_df = pd.DataFrame(history.history) 

#     # save to json:  
#     hist_json_file = "content/History_word2vec/" + str(fold_no) + 'history.json' 
#     with open(hist_json_file, mode='w') as f:
#         hist_df.to_json(f)
            
#     #save model
#     model.save(model_path)
    
  # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
  # Increase fold number
    fold_no = fold_no + 1

  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
422/422 [==============================] - 164s 368ms/step - loss: 0.4565 - accuracy: 0.7954
Epoch 2/5
422/422 [==============================] - 163s 387ms/step - loss: 0.3836 - accuracy: 0.8337
Epoch 3/5
422/422 [==============================] - 158s 375ms/step - loss: 0.3649 - accuracy: 0.8437
Epoch 4/5
422/422 [==============================] - 161s 381ms/step - loss: 0.3505 - accuracy: 0.8498
Epoch 5/5
422/422 [==============================] - 151s 358ms/step - loss: 0.3376 - accuracy: 0.8558
Score for fold 1: loss of 0.3423159718513489; accuracy of 86.12943291664124%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
422/422 [==============================] - 153s 342ms/step - loss: 0.4493 - accuracy: 0.7981
Epoch 2/5
422/422 [==============================] - 144s 341ms/step - loss: 0.3824 - accuracy: 0.8329
Epoch 3/

# Bi-LSTM(word2vec)

In [ ]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=num_folds, shuffle=True)

# K-fold Cross Validation model evaluation
fold_no = 1
for train, test in kfold.split(inputs, targets):

  # Define the model architecture
    model=Sequential()
    embedding_layer=Embedding(vocab_size,
                        EMBEDDING_DIM,
                        embeddings_initializer=Constant(embedding_matrix),
                        input_length=60,
                        trainable=False)  
    model.add(embedding_layer)
    model.add(Bidirectional(LSTM(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
#     model.add(Bidirectional(GRU(units=128, return_sequences = True, dropout=0.25, recurrent_dropout=0.25)))
#     model.add(attention())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1,activation='sigmoid'))
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
  # model.summary()

  # Generate a print
    print('------------------------------------------------------------------------')
    print(f'Training for fold {fold_no} ...')

  # Fit data to model
    history = model.fit(inputs[train], targets[train],
              batch_size=batch_size,
              epochs=no_epochs,
              verbose=verbosity)
#     model_path = "content/models_word2vec/" + str(fold_no) + "model"
    
#     # convert the history.history dict to a pandas DataFrame:     
#     hist_df = pd.DataFrame(history.history) 

#     # save to json:  
#     hist_json_file = "content/History_word2vec/" + str(fold_no) + 'history.json' 
#     with open(hist_json_file, mode='w') as f:
#         hist_df.to_json(f)
            
#     #save model
#     model.save(model_path)
    
  # Generate generalization metrics
    scores = model.evaluate(inputs[test], targets[test], verbose=0)
    print(f'Score for fold {fold_no}: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of {scores[1]*100}%')
    acc_per_fold.append(scores[1] * 100)
    loss_per_fold.append(scores[0])
  # Increase fold number
    fold_no = fold_no + 1

  # == Provide average scores ==
print('------------------------------------------------------------------------')
print('Score per fold')
for i in range(0, len(acc_per_fold)):
    print('------------------------------------------------------------------------')
    print(f'> Fold {i+1} - Loss: {loss_per_fold[i]} - Accuracy: {acc_per_fold[i]}%')
print('------------------------------------------------------------------------')
print('Average scores for all folds:')
print(f'> Accuracy: {np.mean(acc_per_fold)} (+- {np.std(acc_per_fold)})')
print(f'> Loss: {np.mean(loss_per_fold)}')
print('------------------------------------------------------------------------')

------------------------------------------------------------------------
Training for fold 1 ...
Epoch 1/5
422/422 [==============================] - 321s 722ms/step - loss: 0.4529 - accuracy: 0.7921
Epoch 2/5
422/422 [==============================] - 306s 726ms/step - loss: 0.3916 - accuracy: 0.8279
Epoch 3/5
422/422 [==============================] - 305s 722ms/step - loss: 0.3707 - accuracy: 0.8399
Epoch 4/5
422/422 [==============================] - 304s 721ms/step - loss: 0.3527 - accuracy: 0.8484
Epoch 5/5
422/422 [==============================] - 305s 722ms/step - loss: 0.3396 - accuracy: 0.8518
Score for fold 1: loss of 0.34680941700935364; accuracy of 85.84110736846924%
------------------------------------------------------------------------
Training for fold 2 ...
Epoch 1/5
422/422 [==============================] - 318s 717ms/step - loss: 0.4493 - accuracy: 0.7971
Epoch 2/5
422/422 [==============================] - 303s 719ms/step - loss: 0.3912 - accuracy: 0.8294
Epoch 3